<a href="https://colab.research.google.com/github/ftrbrum/Amazon_Vine_Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:15 http://ppa.launchpad.net/grap

In [ ]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-08-09 01:34:10--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.2’

postgresql-42.2.16. 100%[===================>] 979.38K  1.21MB/s    in 0.8s    

2021-08-09 01:34:12 (1.21 MB/s) - ‘postgresql-42.2.16.jar.2’ saved [1002883/1002883]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Sports_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   48945260|R1WBPB8MDCCN8F|B012P7UPSM|     409940130|Chicago Blackhawk...|          Sports|          5|            0|          0|   N|                N|   LOVE IT. 6 stars!|Bought this last ...| 2015-08-31|
|         US|    5782091|R32M0YEWV77XG8|B001GQ3VHG|     657746679|Copag Poker Size ...|          Sports|          5|    

In [ ]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
vine_df = vine_df.withColumn("star_rating", vine_df.star_rating.cast("int")).dropDuplicates(['review_id']).dropna()
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R10009GL6PAAAW|          5|            0|          0|   N|                Y|
|R1006JQWUSRBNN|          5|            2|          2|   N|                Y|
|R1008DD0C9NX9Z|          5|            5|          5|   N|                Y|
|R100AR6CZ42VWH|          5|            0|          0|   N|                Y|
|R1010416UKW5ZU|          5|            2|          2|   N|                Y|
|R1010DZNIMPZZ2|          5|            0|          0|   N|                Y|
|R1018JS8BGMK34|          5|            0|          0|   N|                Y|
|R101HDV705HBFJ|          5|            0|          0|   N|                Y|
|R101LB1X1YF92F|          4|            0|          0|   N|                Y|
|R101LHAQE0WOK4|          5|            0|          1|   N|     

In [ ]:
# Step 1
# filter data and create a new dataframe to retrieve all rows where total_votes is = or > 20
vote_count_20 = vine_df.filter("total_votes >=20")
vote_count_20.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R101U43115T1IP|          5|           36|         39|   N|                Y|
|R10MQKQ3M27SNK|          5|           22|         22|   N|                Y|
| R10S42W67DZT1|          3|           22|         24|   N|                Y|
|R10Y6J50XMLAJB|          4|           44|         52|   N|                N|
|R112KRQ78XAFUW|          5|          128|        141|   N|                Y|
|R116R6PQGUR61W|          1|           23|         32|   N|                Y|
|R11YU1QNX8SBZL|          1|            0|         26|   N|                Y|
|R11ZZYWTLNV17H|          5|           66|         69|   N|                Y|
|R12E7J286GGM6W|          2|           40|         44|   N|                N|
|R12FCXY18H5ZSN|          4|           20|         22|   N|     

In [ ]:
# Step 2
# filter step 1 and create a new dataframe to retrieve all rows where helpful_votes divided by total_votes is = or > 50%
vote_count_50 = vote_count_20.filter(vote_count_20["helpful_votes"]/vote_count_20["total_votes"]>=0.5)
vote_count_50.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R101U43115T1IP|          5|           36|         39|   N|                Y|
|R10MQKQ3M27SNK|          5|           22|         22|   N|                Y|
| R10S42W67DZT1|          3|           22|         24|   N|                Y|
|R10Y6J50XMLAJB|          4|           44|         52|   N|                N|
|R112KRQ78XAFUW|          5|          128|        141|   N|                Y|
|R116R6PQGUR61W|          1|           23|         32|   N|                Y|
|R11ZZYWTLNV17H|          5|           66|         69|   N|                Y|
|R12E7J286GGM6W|          2|           40|         44|   N|                N|
|R12FCXY18H5ZSN|          4|           20|         22|   N|                Y|
|R12X4DD4YL62CR|          4|           99|        110|   N|     

In [ ]:
# Step 3
# filter step 2 and create new dataframe that retrieves all rows where review was written as part of the vine program (paid), vine == 'Y'
vine_program_y = vote_count_50.filter(vote_count_50["vine"] =='Y')
vine_program_y.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3SJHBRUSRAYS6|          5|           20|         23|   Y|                N|
|R3FK059WUSXX5U|          4|           20|         25|   Y|                N|
|R2558BMKF7HK0D|          1|          720|        928|   Y|                N|
| R2R2WU76KCCCL|          5|           19|         21|   Y|                N|
|R17V0U2RS9UGBY|          4|          187|        200|   Y|                N|
|R2OTHTVVK4RZXI|          4|           50|         60|   Y|                N|
|R3IJWNLDMHOSW8|          5|           31|         33|   Y|                N|
|R1Z192NHYX1PWT|          4|           38|         42|   Y|                N|
| RX8TSC5PKNMB9|          5|           27|         29|   Y|                N|
|R329N0ZWG4TYHA|          4|           29|         30|   Y|     

In [ ]:
# Step 4
# filter step 2 and create new dataframe that retrieves all rows where review was written as NOT part of the vine program (unpaid), vine == 'N'
vine_program_n = vote_count_50.filter(vote_count_50["vine"] =='N')
vine_program_n.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R101U43115T1IP|          5|           36|         39|   N|                Y|
|R10MQKQ3M27SNK|          5|           22|         22|   N|                Y|
| R10S42W67DZT1|          3|           22|         24|   N|                Y|
|R10Y6J50XMLAJB|          4|           44|         52|   N|                N|
|R112KRQ78XAFUW|          5|          128|        141|   N|                Y|
|R116R6PQGUR61W|          1|           23|         32|   N|                Y|
|R11ZZYWTLNV17H|          5|           66|         69|   N|                Y|
|R12E7J286GGM6W|          2|           40|         44|   N|                N|
|R12FCXY18H5ZSN|          4|           20|         22|   N|                Y|
|R12X4DD4YL62CR|          4|           99|        110|   N|     

In [ ]:
# step 5 A
# Determine the total number of reviews, 
reviews_total = vote_count_50.select("star_rating").count()
reviews_total

61948

In [ ]:
# Step 5 B
# the number of 5-star reviews 
reviews_5_star = vote_count_50.filter(vine_df["star_rating"] == "5").count()
reviews_5_star

32804

In [ ]:
# Step 5 C
# paid and unpaid reviews
reviews_5_star_paid = vine_program_y.filter(vote_count_50["star_rating"] == "5").count()
reviews_5_star_unpaid = vine_program_n.filter(vine_df["star_rating"] == "5").count()

In [ ]:
# Step 5 D
# , and the percentage of 5-star reviews paid
reviews_percent_5_star_paid = reviews_5_star_paid/reviews_total * 100
reviews_percent_5_star_paid

0.2243817395234713

In [ ]:
# Step 5 E
#  and the percentage of 5-star reviews unpaid
reviews_percent_5_star_unpaid = reviews_5_star_unpaid/reviews_total * 100
reviews_percent_5_star_unpaid

52.72970878801575